In [1]:
import os
from xml.etree import ElementTree
import matplotlib.pyplot as plt
import numpy as np
from skimage.io import imshow, imread
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn import ensemble
from sklearn.metrics import accuracy_score
import cv2
from sklearn.svm import SVC
from PIL import Image

# Creating the database

In [2]:
annotations_path = "F:\Projects\MEDH_projects\ExtractingImageFeaturesHSV\VOCdevkit\VOC2007\Annotations"
images_path = "F:\Projects\MEDH_projects\ExtractingImageFeaturesHSV\VOCdevkit\VOC2007\JPEGImages"

In [3]:
tags = []
images = []

## Creating dict for object-tag one label

In [4]:
%%time

dataset_dict = {}

for file in os.listdir(annotations_path):
    file_path = os.path.join(annotations_path, file)
    print(file_path)
    dom = ElementTree.parse(file_path)
    objects = dom.findall("object")
    for object in objects:
        if len(objects) == 1:
            object_label = object.find("name").text
        elif object.find("truncated").text == "1":
            if object == objects[-1]:
                object_label = object.find("name").text
            continue
        else:
            object_label = object.find("name").text
            break
    dataset_dict[file] = object_label

F:\Projects\MEDH_projects\ExtractingImageFeaturesHSV\VOCdevkit\VOC2007\Annotations\000001.xml
F:\Projects\MEDH_projects\ExtractingImageFeaturesHSV\VOCdevkit\VOC2007\Annotations\000002.xml
F:\Projects\MEDH_projects\ExtractingImageFeaturesHSV\VOCdevkit\VOC2007\Annotations\000003.xml
F:\Projects\MEDH_projects\ExtractingImageFeaturesHSV\VOCdevkit\VOC2007\Annotations\000004.xml
F:\Projects\MEDH_projects\ExtractingImageFeaturesHSV\VOCdevkit\VOC2007\Annotations\000005.xml
F:\Projects\MEDH_projects\ExtractingImageFeaturesHSV\VOCdevkit\VOC2007\Annotations\000006.xml
F:\Projects\MEDH_projects\ExtractingImageFeaturesHSV\VOCdevkit\VOC2007\Annotations\000007.xml
F:\Projects\MEDH_projects\ExtractingImageFeaturesHSV\VOCdevkit\VOC2007\Annotations\000008.xml
F:\Projects\MEDH_projects\ExtractingImageFeaturesHSV\VOCdevkit\VOC2007\Annotations\000009.xml
F:\Projects\MEDH_projects\ExtractingImageFeaturesHSV\VOCdevkit\VOC2007\Annotations\000010.xml
F:\Projects\MEDH_projects\ExtractingImageFeaturesHSV\VOCdevk

## Creating dict for multilabel

In [5]:
%%time

dataset_dict = {}

for file in os.listdir(annotations_path):
    file_path = os.path.join(annotations_path, file)
    print(file_path)
    dom = ElementTree.parse(file_path)
    objects = dom.findall("object")
    objects_classes = []
    for object in objects:
        if object.find("name").text not in objects_classes:
            objects_classes.append(object.find("name").text)
        else:
            continue
    dataset_dict[file] = objects_classes

F:\Projects\MEDH_projects\ExtractingImageFeaturesHSV\VOCdevkit\VOC2007\Annotations\000001.xml
F:\Projects\MEDH_projects\ExtractingImageFeaturesHSV\VOCdevkit\VOC2007\Annotations\000002.xml
F:\Projects\MEDH_projects\ExtractingImageFeaturesHSV\VOCdevkit\VOC2007\Annotations\000003.xml
F:\Projects\MEDH_projects\ExtractingImageFeaturesHSV\VOCdevkit\VOC2007\Annotations\000004.xml
F:\Projects\MEDH_projects\ExtractingImageFeaturesHSV\VOCdevkit\VOC2007\Annotations\000005.xml
F:\Projects\MEDH_projects\ExtractingImageFeaturesHSV\VOCdevkit\VOC2007\Annotations\000006.xml
F:\Projects\MEDH_projects\ExtractingImageFeaturesHSV\VOCdevkit\VOC2007\Annotations\000007.xml
F:\Projects\MEDH_projects\ExtractingImageFeaturesHSV\VOCdevkit\VOC2007\Annotations\000008.xml
F:\Projects\MEDH_projects\ExtractingImageFeaturesHSV\VOCdevkit\VOC2007\Annotations\000009.xml
F:\Projects\MEDH_projects\ExtractingImageFeaturesHSV\VOCdevkit\VOC2007\Annotations\000010.xml
F:\Projects\MEDH_projects\ExtractingImageFeaturesHSV\VOCdevk

In [6]:
dataset_dict

{'000001.xml': ['dog', 'person'],
 '000002.xml': ['train'],
 '000003.xml': ['sofa', 'chair'],
 '000004.xml': ['car'],
 '000005.xml': ['chair'],
 '000006.xml': ['pottedplant', 'diningtable', 'chair'],
 '000007.xml': ['car'],
 '000008.xml': ['chair'],
 '000009.xml': ['horse', 'person'],
 '000010.xml': ['horse', 'person'],
 '000011.xml': ['cat'],
 '000012.xml': ['car'],
 '000013.xml': ['cow'],
 '000014.xml': ['bus', 'car', 'person'],
 '000015.xml': ['bicycle'],
 '000016.xml': ['bicycle'],
 '000017.xml': ['person', 'horse'],
 '000018.xml': ['dog'],
 '000019.xml': ['cat'],
 '000020.xml': ['car'],
 '000021.xml': ['dog', 'person'],
 '000022.xml': ['horse', 'person'],
 '000023.xml': ['bicycle', 'person'],
 '000024.xml': ['train'],
 '000025.xml': ['cow', 'person'],
 '000026.xml': ['car'],
 '000027.xml': ['person'],
 '000028.xml': ['cat'],
 '000029.xml': ['dog'],
 '000030.xml': ['bicycle', 'person'],
 '000031.xml': ['train'],
 '000032.xml': ['aeroplane', 'person'],
 '000033.xml': ['aeroplane'],


## Reading images and creating image and tag lists

In [7]:
for _, item in dataset_dict.items():
    tags.append(item)

In [8]:
for key in dataset_dict:
    file_path = os.path.join(images_path, key.replace('.xml', '.jpg'))
    image = Image.open(file_path)
    new_img = image.resize((256, 256))
    img = np.array(new_img)
    images.append(img)

In [9]:
images

[array([[[  1,   1,   0],
         [  1,   1,   0],
         [  1,   1,   0],
         ...,
         [  3,   5,   4],
         [  3,   5,   4],
         [  1,   3,   2]],
 
        [[  1,   1,   0],
         [  1,   1,   0],
         [  1,   1,   0],
         ...,
         [  4,   6,   5],
         [  3,   5,   4],
         [  2,   2,   2]],
 
        [[  1,   1,   0],
         [  1,   1,   0],
         [  1,   1,   0],
         ...,
         [  6,   7,   7],
         [  3,   4,   4],
         [  2,   2,   2]],
 
        ...,
 
        [[ 16,  37,  77],
         [ 23,  40,  75],
         [ 35,  50,  82],
         ...,
         [ 40,  71, 137],
         [ 33,  62, 128],
         [ 31,  56, 123]],
 
        [[ 33,  44,  85],
         [ 32,  47,  82],
         [ 15,  31,  66],
         ...,
         [ 31,  63, 123],
         [ 25,  55, 116],
         [ 29,  55, 120]],
 
        [[ 12,  22,  60],
         [ 35,  49,  83],
         [ 30,  46,  81],
         ...,
         [ 38,  69, 129],
  

In [10]:
tags

[['dog', 'person'],
 ['train'],
 ['sofa', 'chair'],
 ['car'],
 ['chair'],
 ['pottedplant', 'diningtable', 'chair'],
 ['car'],
 ['chair'],
 ['horse', 'person'],
 ['horse', 'person'],
 ['cat'],
 ['car'],
 ['cow'],
 ['bus', 'car', 'person'],
 ['bicycle'],
 ['bicycle'],
 ['person', 'horse'],
 ['dog'],
 ['cat'],
 ['car'],
 ['dog', 'person'],
 ['horse', 'person'],
 ['bicycle', 'person'],
 ['train'],
 ['cow', 'person'],
 ['car'],
 ['person'],
 ['cat'],
 ['dog'],
 ['bicycle', 'person'],
 ['train'],
 ['aeroplane', 'person'],
 ['aeroplane'],
 ['train'],
 ['person', 'diningtable'],
 ['dog'],
 ['dog'],
 ['bicycle', 'motorbike', 'person'],
 ['tvmonitor'],
 ['bird'],
 ['tvmonitor', 'person'],
 ['train'],
 ['person'],
 ['chair', 'cat'],
 ['tvmonitor'],
 ['bird'],
 ['car', 'chair'],
 ['bird', 'person'],
 ['cat'],
 ['chair', 'bottle', 'person'],
 ['motorbike', 'person'],
 ['pottedplant'],
 ['cat'],
 ['bus'],
 ['motorbike', 'person'],
 ['horse'],
 ['bird'],
 ['person', 'motorbike'],
 ['tvmonitor', 'pers

# Extracting the features

In [11]:
def hue_histogram(image):

    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    hue, _, _ = cv2.split(hsv)
    hist = cv2.calcHist([hue], [0], None, [180], [0, 180])
    hist = cv2.normalize(hist, hist, 0, 1, cv2.NORM_MINMAX)

    return hist.ravel()

In [12]:
images_hue = []
for image in images:
    images_hue.append(hue_histogram(image))

In [13]:
length_dataset = len(images_hue)

In [14]:
images_hue = np.asarray(images_hue)

# Data splitting

In [15]:
mlp = MultiLabelBinarizer()

In [16]:
labels = mlp.fit_transform(tags)

In [17]:
labels.shape

(9963, 20)

In [18]:
train_ds, test_ds, train_tags, test_tags = train_test_split(images_hue, labels, test_size=0.2, shuffle=True)

# Dimensionality Reduction

# Training

In [41]:
model = ensemble.RandomForestClassifier()

In [19]:
model = SVC(C=100, kernel='linear')

In [46]:
train_ds.shape

(7970, 180)

In [47]:
train_tags.shape

(7970, 20)

In [20]:
multi_model = MultiOutputClassifier(model, n_jobs=-1)
multi_model.fit(train_ds, train_tags)

OSError: [Errno 22] Invalid argument

# Prediction

In [ ]:
predicts = multi_model.predict(test_ds)

accuracy = accuracy_score(y_true=test_tags, y_pred = predicts)
print("Accuracy = ", accuracy)